# Rheumatoid Arthritis Case Control Association Testing

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import harmonypy as hm
import pp, cna, os, pickle
pp.umapprops['s'] = 0.4
import multianndata as mad
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as st
fig_dir = '/data/srlab/lrumker/MCSC_Project/cna-prs/figs/'
src_folder = "/data/srlab/lrumker/datasets/onek1k/pheno/"
res_folder = "/data/srlab/lrumker/MCSC_Project/cna-prs/results/sc_objects/"
np.random.seed(0)

In [2]:
# From Okabe & Ito, "colorblind-friendly" palette 
from matplotlib.colors import LinearSegmentedColormap
color_dict = {'orange': '#E69F00', 'skyblue': '#56B4E9', 'green': '#009E73', 
              'yellow': '#F0E442', 'purple': '#CC79A7', 'oceanblue': '#0072B2', 
              'vermillion': '#E63d00'} # O&I use '#D55E00'

In [3]:
# Subscript in text mode not math mode (avoids italics)
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

## Build single-cell object for case-control cohort

In [4]:
for celltype in ["NK", "B", "T", "allcells", "Myeloid"]:
    print(celltype)
    np.random.seed(0)
    d = sc.read_h5ad(src_folder+celltype+"_expr.h5ad")

    # Remove individuals without documented clinical information
    meta = pd.read_csv(src_folder+"sample_meta.csv", index_col = 0)
    meta_clin = meta.drop(columns = meta.columns[pd.isna(meta).sum(axis=0)<400]) # 32 clinical variables
    rm_ids = meta_clin.index[pd.isna(meta_clin).sum(axis=1)==meta_clin.shape[1]]
    meta = meta.drop(index=rm_ids)

    # Remove individuals with non-RA autoimmune disease
    meta = meta.loc[meta.Autoimmune_Disease==0,:]
    meta = meta.loc[meta.Diabetes_type1==0,:]
    meta = meta.loc[meta.UlcerativeColitis==0,:]
    meta = meta.loc[meta.Autoimmune_Disease_Other=="N",:]
    keep_ids = meta.index
    keep_cells = np.repeat(False, d.obs.shape[0])
    for sel_donor in keep_ids:
        keep_cells[np.where(d.obs.individual.values==sel_donor)] = True
    d = d[keep_cells,:]
    print("Keeping "+str(len(np.unique(d.obs.individual)))+" samples with RA or known absence of non-RA autoimmune disease")

    # Retain only samples with at least 25 cells
    cellcount = pd.DataFrame(d.obs.individual.value_counts())
    cellcount.columns = ['n_cells']
    keep_ids = cellcount.index[cellcount.n_cells>=25]
    meta = meta.loc[keep_ids,:]
    keep_cells = np.repeat(False, d.obs.shape[0])
    for sel_donor in keep_ids:
        keep_cells[np.where(d.obs.individual.values==sel_donor)] = True

    d = d[keep_cells,:]
    print("Keeping "+str(len(np.unique(d.obs.individual)))+" samples with at least 25 cells")

    # Downsample controls at random to a 50% case rate
    n_RA = np.sum(meta.Rheumatoid_arthritis==1)
    kept_ids = d.obs.individual.value_counts().index
    meta = meta.loc[kept_ids,:]
    np.random.seed(0)
    candidates = meta.index[meta.Rheumatoid_arthritis==0]
    controls = np.random.choice(candidates,n_RA,replace = False)

    keep_ids = np.concatenate((np.array(controls), np.array(meta.index[meta.Rheumatoid_arthritis==1])))
    keep_cells = np.repeat(False, d.obs.shape[0])
    for sel_donor in keep_ids:
        keep_cells[np.where(d.obs.individual.values==sel_donor)] = True

    d = d[keep_cells,:]
    print("Keeping "+str(len(np.unique(d.obs.individual)))+" samples after downsampling controls to 50% case rate")

    # Remove all HLA- genes (21)
    d.var['HLA'] = ['HLA-' in d.var.index[i] for i in np.arange(d.var.shape[0])]
    d = d[:,~d.var.HLA.values]

    # Remove cell cycle genes
    cc_genes = ["MCM5","PCNA","TYMS","FEN1","MCM2","MCM4", "RRM1","UNG","GINS2","MCM6",
                "CDCA7","DTL","PRIM1","UHRF1","MLF1IP","HELLS","RFC2","RPA2","NASP", 
                "RAD51AP1","GMNN","WDR76","SLBP","CCNE2","UBR7","POLD3","MSH2","ATAD2",
                "RAD51","RRM2", "CDC45", "CDC6", "EXO1", "TIPIN", "DSCC1", "BLM", "CASP8AP2",
                "USP1","CLSPN","POLA1","CHAF1B","BRIP1","E2F8","HMGB2","CDK1","NUSAP1","UBE2C",
                "BIRC5","TPX2","TOP2A","NDC80","CKS2","NUF2","CKS1B","MKI67","TMPO","CENPF",
                "TACC3","FAM64A","SMC4","CCNB2","CKAP2L","CKAP2","AURKB","BUB1","KIF11",
                "ANP32E","TUBB4B","GTSE1","KIF20B","HJURP","CDCA3","HN1", "CDC20", "TTK",
                "CDC25C", "KIF2C", "RANGAP1", "NCAPD2", "DLGAP5", "CDCA2", "CDCA8", "ECT2", 
                "KIF23", "HMMR", "AURKA", "PSRC1", "ANLN", "LBR", "CKAP5", "CENPE", "CTCF",
                "NEK2","G2E3","GAS2L3","CBX5","CENPA"]
    d.var['CC'] = [d.var.index[i] in cc_genes for i in np.arange(d.var.shape[0])]
    d = d[:,~d.var.CC.values]

    # Remove hemoglobin genes (polymorphic)
    hb_genes = ['HBB', 'HBA2', 'HBD', 'HBA1']
    d.var['HB'] = [d.var.index[i] in hb_genes for i in np.arange(d.var.shape[0])]
    d = d[:,~d.var.HB.values]

    # Remove platelet genes
    plt_genes = ['PF4', 'PPBP']
    d.var['Plt'] = [d.var.index[i] in plt_genes for i in np.arange(d.var.shape[0])]
    d = d[:,~d.var.Plt.values]

    sc.pp.normalize_total(d, target_sum=1e4) #normalize expr
    sc.pp.log1p(d) #logarithmize

    # variable gene selection
    min_disp = {'Myeloid':0.72, 'B':0.58, 'NK': 0.55, 'T': 0.47, 'allcells': 0.42}
    sc.pp.highly_variable_genes(d, min_disp=min_disp[celltype])
    np.sum(d.var.highly_variable)

    high_dispersion = d.var.dispersions_norm > 11
    d.var.loc[high_dispersion, 'highly_variable'] = False

    d = d[:, d.var.highly_variable]

    sc.pp.scale(d, max_value=10) # Scale each gene to unit variance
    sc.tl.pca(d, svd_solver='arpack') # PCA

    # Harmonize over batch (theta of 2 is default, if >1 batch variable, thetas should sum to 1)
    if celltype == "allcells":
        ho = hm.run_harmony(d.obsm['X_pca'][:,:20], d.obs, ['pool'], max_iter_harmony = 50, theta = 2)
    else:
        # sel sigma 0.2 > default of 0.1 --> encourages softer clustering b/c all one major type
        ho = hm.run_harmony(d.obsm['X_pca'][:,:20], d.obs, ['pool'], 
                            nclust = 50, sigma = 0.2, max_iter_harmony = 50, theta = 2)
    d.obsm['harmpca'] = ho.Z_corr.T

    print("graph")
    sc.pp.neighbors(d, use_rep = 'harmpca') # graph    
    print("umap")
    sc.tl.umap(d) # umap 

    # Load cell metadata
    cell_meta = pd.read_csv(src_folder+"cell_meta.csv", index_col = 0)
    cell_meta['batch'] = cell_meta.pool_number.values
    d.obs['id'] = d.obs.individual.values
    d.obs['preQC_celltype'] = d.obs['predicted.celltype.l2'].values

    d.obs = d.obs.loc[:,['id', 'i_RawExpr', 'majortype', 'celltype', 'ref_UMAP1', 'ref_UMAP2', 'preQC_celltype']]
    d.obs = d.obs.join(cell_meta.loc[:,['nCount_RNA', 'nFeature_RNA', 'pool', 'percent.mt', 'batch',
            'sex', 'age', 'indiv_barcode']])

    # make anndata object
    d = mad.MultiAnnData(d, sampleid='id')

    # aggregate sample metadata imported per-cell
    d.obs_to_sample(['sex', 'age', 'batch'])
    d.samplem['sex_M'] = (d.samplem.sex==1)*1 # From 1 vs 2 to boolean
    d.samplem = d.samplem.drop(columns = ['sex'])

    # add other clinical metadata
    d.samplem = d.samplem.join(meta.loc[:,['gPC1', 'gPC2', 'gPC3', 'gPC4', 'gPC5', 'gPC6']])
    d.samplem = d.samplem.join(meta.loc[:,'Rheumatoid_arthritis'])

    categorical = ['Autoimmune_Disease_Other', 'Ca_Type', 'Eye_DiseaseType', 'Other_Disease', 'Other_Meds']
    for attribute in d.samplem.columns:
        if attribute not in categorical: d.samplem[attribute] = d.samplem[attribute].values.tolist()
    for attribute in d.samplem.columns:
        if attribute in categorical: d.samplem[attribute] = d.samplem[attribute].values.astype(str).tolist()

    # build NAM, compute NAM-PCs corrected for batch and covariates
    covs = ['age', 'sex_M']
    cna.tl.nam(d, batches=d.samplem.batch, covs=d.samplem[covs], ks=[d.samplem.shape[0]])

    # save data objects
    d.write("/data/srlab/lrumker/MCSC_Project/cna-prs/results/sc_objects/1K1K_casecontrol_"+celltype+".h5ad")

NK


Trying to set attribute `.var` of view, copying.
Trying to set attribute `.var` of view, copying.


Keeping 503 samples with RA or known absence of non-RA autoimmune disease
Keeping 483 samples with at least 25 cells
Keeping 30 samples after downsampling controls to 50% case rate


Trying to set attribute `.var` of view, copying.
Trying to set attribute `.var` of view, copying.
/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:845: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
2023-11-05 18:39:18,125 - harmonypy - INFO - Iteration 1 of 50
2023-11-05 18:39:18,753 - harmonypy - INFO - Iteration 2 of 50
2023-11-05 18:39:19,386 - harmonypy - INFO - Iteration 3 of 50
2023-11-05 18:39:20,019 - harmonypy - INFO - Iteration 4 of 50
2023-11-05 18:39:20,232 - harmonypy - INFO - Iteration 5 of 50
2023-11-05 18:39:20,474 - harmonypy - INFO - Iteration 6 of 50
2023-11-05 18:39:20,878 - harmonypy - INFO - Iteration 7 of 50
2023-11-05 18:39:21,340 - harmonypy - INFO - Iteration 8 of 50
2023-11-05 18:3

graph
umap


...

Trying to set attribute `.var` of view, copying.
Trying to set attribute `.var` of view, copying.
/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:845: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
2023-11-05 18:47:16,429 - harmonypy - INFO - Iteration 1 of 50
2023-11-05 18:47:16,712 - harmonypy - INFO - Iteration 2 of 50
2023-11-05 18:47:16,833 - harmonypy - INFO - Iteration 3 of 50
2023-11-05 18:47:17,050 - harmonypy - INFO - Iteration 4 of 50
2023-11-05 18:47:17,209 - harmonypy - INFO - Iteration 5 of 50
2023-11-05 18:47:17,477 - harmonypy - INFO - Iteration 6 of 50
2023-11-05 18:47:17,620 - harmonypy - INFO - Iteration 7 of 50
2023-11-05 18:47:17,859 - harmonypy - INFO - Iteration 8 of 50
2023-11-05 18:4

graph
umap


/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['id' 'majortype' 'celltype' 'preQC_celltype' 'pool' 'indiv_barcode']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'pool' as categorical


## Test for association

In [5]:
nnull=10000
for celltype in ["B", "NK", "allcells", "Myeloid", "T"]:
    print(celltype)
    np.random.seed(0)

    d = cna.read("/data/srlab/lrumker/MCSC_Project/cna-prs/results/sc_objects/1K1K_casecontrol_"+celltype+".h5ad")

    # Select k based on cumulative % variance explained
    max_nampc = []
    for cum_var_exp in [0.50, 0.80]:
        max_nampc.append(np.min(np.where(np.cumsum(d.uns['NAM_varexp'])>cum_var_exp)[0]))
    print(max_nampc)
    res = cna.tl.association(d, d.samplem.Rheumatoid_arthritis, batches = d.samplem.batch, 
                     covs = d.samplem[['age', 'sex_M']], ks = max_nampc, Nnull=nnull)
    print(res.p)

    vargene_cors = []
    for i_gene in np.arange(d.var.shape[0]):
        vargene_cors.append(np.corrcoef(d.X[res.kept, i_gene], res.ncorrs)[0,1])
    res.vargene_cors=pd.DataFrame({'gene':d.var.index, 'cor': vargene_cors})

    res.UMAP1 = d.obsm['X_umap'][res.kept,0]
    res.UMAP2 = d.obsm['X_umap'][res.kept,1]

    pickle.dump(res, open("/data/srlab/lrumker/MCSC_Project/cna-prs/results/RA_casecontrol/1K1K_casecontrol_"+celltype+".p", 'wb'))

B
[3, 11]


/data/srlab/lrumker/MCSC_Project/cna/src/cna/tools/_association.py:61: UserWarning: data supported use of 11 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\
/data/srlab/lrumker/MCSC_Project/cna/src/cna/tools/_association.py:79: UserWarning: global association p-value attained minimal possible value. Consider increasing Nnull
  warnings.warn('global association p-value attained minimal possible value. '+\


9.999000099990002e-05
NK
[4, 13]
9.999000099990002e-05
allcells
[5, 16]
9.999000099990002e-05
Myeloid
[2, 5]
9.999000099990002e-05


/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/numpy/lib/function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/PHShome/lar24/anaconda3/envs/py3_clone/lib/python3.8/site-packages/numpy/lib/function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


T
[4, 15]
9.999000099990002e-05


## Assemble results

In [6]:
all_res = pd.DataFrame({})
for celltype in ["Myeloid", "allcells", "NK", "B", "T"]:
    res = pickle.load(open("/data/srlab/lrumker/MCSC_Project/cna-prs/results/RA_casecontrol/1K1K_casecontrol_"+\
                                celltype+".p", 'rb'))
    d = cna.read("/data/srlab/lrumker/MCSC_Project/cna-prs/results/sc_objects/1K1K_casecontrol_"+celltype+".h5ad")
    new = pd.DataFrame({"Cell Type": [celltype],
                        "N cases": [d.samplem.loc[d.samplem.Rheumatoid_arthritis==1,].shape[0]], 
                        "N conrtrols": [d.samplem.loc[d.samplem.Rheumatoid_arthritis==0,].shape[0]], 
                        "P": [r"$<$1e-4"]})
    all_res = pd.concat([all_res, new])

table_latex = all_res.to_latex(index = False,  escape=False,
              column_format='p{1.8cm}|p{1.8cm}|p{1.8cm}|p{1.8cm}')
table_latex = table_latex.replace("\\\n", "\\ \hline\n") # add lines between rows
with open('/data/srlab/lrumker/MCSC_Project/cna-qtl/tables/supptable.ra_cc_results.tex','w') as tf:
    tf.write(table_latex)
all_res

,Cell Type,N cases,N conrtrols,P
0,Myeloid,9,9,$<$1e-4
0,allcells,16,16,$<$1e-4
0,NK,15,15,$<$1e-4
0,B,15,15,$<$1e-4
0,T,16,16,$<$1e-4
